In [ ]:
from keras.layers import Input, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
pip install split_folders

In [ ]:
import splitfolders
splitfolders.ratio("/content/gdrive/MyDrive/Colab Dataset/chest_xray/train-test", output="/content/gdrive/MyDrive/Colab Dataset/chest_xray/split", seed=1337, ratio=(.70,0.30) ,group_prefix=None)

Copying files: 5840 files [36:11,  2.69 files/s]


In [ ]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights
IMAGE_SIZE = [224, 224]
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
# useful for getting number of output classes
folders = glob('/content/gdrive/MyDrive/Colab Dataset/chest_xray/split/train/*')

In [ ]:
folders

['/content/gdrive/MyDrive/Colab Dataset/chest_xray/split/train/PNEUMONIA',
 '/content/gdrive/MyDrive/Colab Dataset/chest_xray/split/train/NORMAL']

In [ ]:
# our layers - Flattening
x = Flatten()(vgg.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# Use the Image Data Generator to import the images from the dataset
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory("/content/gdrive/MyDrive/Colab Dataset/chest_xray/split/train",
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 4087 images belonging to 2 classes.


In [ ]:
test_set = test_datagen.flow_from_directory("/content/gdrive/MyDrive/Colab Dataset/chest_xray/split/val",
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1753 images belonging to 2 classes.


In [ ]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
128/128 [==============================] - 1458s 11s/step - loss: 0.3644 - accuracy: 0.8486 - val_loss: 0.1916 - val_accuracy: 0.9327
Epoch 2/5
128/128 [==============================] - 98s 770ms/step - loss: 0.1430 - accuracy: 0.9486 - val_loss: 0.1538 - val_accuracy: 0.9412
Epoch 3/5
128/128 [==============================] - 99s 771ms/step - loss: 0.1269 - accuracy: 0.9548 - val_loss: 0.2563 - val_accuracy: 0.9070
Epoch 4/5
128/128 [==============================] - 98s 768ms/step - loss: 0.1400 - accuracy: 0.9488 - val_loss: 0.1069 - val_accuracy: 0.9618
Epoch 5/5
128/128 [==============================] - 99s 772ms/step - loss: 0.0968 - accuracy: 0.9634 - val_loss: 0.1851 - val_accuracy: 0.9361


In [ ]:
# save it as a h5 file

model.save('model_vgg16_70-30.h5')